# Data Anaysis Project part1:

I will follow what I am learning from Google Data Analytics course to complete this project.

* Ask
* Prepare
* Process
* Analyze
* Share
* Act



## Overview of the project:

Analyze housing rental market in Fraser Valley to find out what is hot to real estate investors in this area. Iam going to create the code to analyze dynamically so the investors can apply in their area for their own use.  


## ASK 
    
    Find out what is 'HOT' in the market for investor, so i can find a property for them. Find out supply and demand of the rental housing market and calculate appropreate purchase price. 

#### What are the questions to ask?
Supply side:
* How much is the market rental price?
* What type of the rental is most popular and least popular?
* How many ads per day? Number of supply 
* Is rental price rising or decreasing over time?
* What is the correlation between housing price and rental price?
* From those questions, what is my prediction of the rental market? 

Demand side:
* What kind of propety does people look for rent?
* What is the price range they are looking for?
* What are the purposes of renting. (Who is looking for rent)





## Prepare  

#### Using Craigslist add to collect market data: (Supply side)
Scrape craigslist rental housing pages and store in csv file. 

<a href='https://github.com/rileypredum/East-Bay-Housing-Web-Scrape/blob/master/EB_Apt_Prices_Final.ipynb'>Code: I will use this code and modify for my situation.  </a> <br>
<a href='https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981'>Explaning</a>

## Process

I noticed that the all the values are string, so I will convert to correct data type as I collect the data. 

In [2]:
# Import libralis
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np

from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.display import clear_output

In [7]:
def get_url(city_name, num_post):
    return f'https://{city_name.lower()}.craigslist.org/search/apa?s={num_post}'

In [8]:
# Chilliwack rental housing

response = get(get_url(city_name = 'Abbotsford', num_post = 120))


soup = BeautifulSoup(response.text, 'html.parser')

posts = soup.find_all('li', class_='result-row')
print(type(posts)) # to double check that I got resultSet
print(len(posts)) # to double check I got 120 elements

<class 'bs4.element.ResultSet'>
120


In [40]:
# Find the total number of posts to find the limit of the pagination
city = 'Vancouver'
response = get(get_url(city_name = city, num_post=0))
soup = BeautifulSoup(response.text, 'html.parser')
posts = soup.find_all('li', class_='result-row')
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
print(f"Total number of search result: {results_total}")

# each page has 119 posts so each new page is defined as follows; s=120, s=240, .....
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_date = []
post_cities = []
num_bedroom = []
sqfts = []
prices = []
post_titles = []

for page in pages:
    
    # get request
    response = get(get_url(city_name=city, num_post=page))
    
    sleep(1)
    
    # throw warning for status code that are not 200
    if response.status_code != 200:
        warn(f'Request: {requests}; Status code: {response.status_code}')
        
    soup = BeautifulSoup(response.text, 'html.parser')
    
    posts = soup.find_all('li', class_='result-row')
    
    
    # Extract data 
    for post in posts:
        
        if post.find('span', class_ = 'result-hood') is not None:
            
            # Posting date
            post_datetime = post.find('time', class_='result-date')['datetime']
            post_date.append(post_datetime)
            
            # Neighbourhoods
            post_city = post.find('span', class_='result-hood').text.strip('( )')
            post_cities.append(post_city)
            
            # title text
            post_title = post.find('a', class_='result-title hdrlnk').text
            post_titles.append(post_title)
            
            # Post price in integer
            price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
            prices.append(price)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    num_bedroom.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    num_bedroom.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                num_bedroom.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    num_bedroom.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
        iterations += 1
        print("Page" + str(iterations) + "Scraped Successfully!")   

Total number of search result: 3000
Page1Scraped Successfully!
Page2Scraped Successfully!
Page3Scraped Successfully!
Page4Scraped Successfully!
Page5Scraped Successfully!
Page6Scraped Successfully!
Page7Scraped Successfully!
Page8Scraped Successfully!
Page9Scraped Successfully!
Page10Scraped Successfully!
Page11Scraped Successfully!
Page12Scraped Successfully!
Page13Scraped Successfully!
Page14Scraped Successfully!
Page15Scraped Successfully!
Page16Scraped Successfully!
Page17Scraped Successfully!
Page18Scraped Successfully!
Page19Scraped Successfully!
Page20Scraped Successfully!
Page21Scraped Successfully!
Page22Scraped Successfully!
Page23Scraped Successfully!
Page24Scraped Successfully!
Page25Scraped Successfully!
Page26Scraped Successfully!
Page27Scraped Successfully!
Page28Scraped Successfully!
Page29Scraped Successfully!
Page30Scraped Successfully!
Page31Scraped Successfully!
Page32Scraped Successfully!
Page33Scraped Successfully!
Page34Scraped Successfully!
Page35Scraped Success

Page361Scraped Successfully!
Page362Scraped Successfully!
Page363Scraped Successfully!
Page364Scraped Successfully!
Page365Scraped Successfully!
Page366Scraped Successfully!
Page367Scraped Successfully!
Page368Scraped Successfully!
Page369Scraped Successfully!
Page370Scraped Successfully!
Page371Scraped Successfully!
Page372Scraped Successfully!
Page373Scraped Successfully!
Page374Scraped Successfully!
Page375Scraped Successfully!
Page376Scraped Successfully!
Page377Scraped Successfully!
Page378Scraped Successfully!
Page379Scraped Successfully!
Page380Scraped Successfully!
Page381Scraped Successfully!
Page382Scraped Successfully!
Page383Scraped Successfully!
Page384Scraped Successfully!
Page385Scraped Successfully!
Page386Scraped Successfully!
Page387Scraped Successfully!
Page388Scraped Successfully!
Page389Scraped Successfully!
Page390Scraped Successfully!
Page391Scraped Successfully!
Page392Scraped Successfully!
Page393Scraped Successfully!
Page394Scraped Successfully!
Page395Scraped

Page721Scraped Successfully!
Page722Scraped Successfully!
Page723Scraped Successfully!
Page724Scraped Successfully!
Page725Scraped Successfully!
Page726Scraped Successfully!
Page727Scraped Successfully!
Page728Scraped Successfully!
Page729Scraped Successfully!
Page730Scraped Successfully!
Page731Scraped Successfully!
Page732Scraped Successfully!
Page733Scraped Successfully!
Page734Scraped Successfully!
Page735Scraped Successfully!
Page736Scraped Successfully!
Page737Scraped Successfully!
Page738Scraped Successfully!
Page739Scraped Successfully!
Page740Scraped Successfully!
Page741Scraped Successfully!
Page742Scraped Successfully!
Page743Scraped Successfully!
Page744Scraped Successfully!
Page745Scraped Successfully!
Page746Scraped Successfully!
Page747Scraped Successfully!
Page748Scraped Successfully!
Page749Scraped Successfully!
Page750Scraped Successfully!
Page751Scraped Successfully!
Page752Scraped Successfully!
Page753Scraped Successfully!
Page754Scraped Successfully!
Page755Scraped

Page1081Scraped Successfully!
Page1082Scraped Successfully!
Page1083Scraped Successfully!
Page1084Scraped Successfully!
Page1085Scraped Successfully!
Page1086Scraped Successfully!
Page1087Scraped Successfully!
Page1088Scraped Successfully!
Page1089Scraped Successfully!
Page1090Scraped Successfully!
Page1091Scraped Successfully!
Page1092Scraped Successfully!
Page1093Scraped Successfully!
Page1094Scraped Successfully!
Page1095Scraped Successfully!
Page1096Scraped Successfully!
Page1097Scraped Successfully!
Page1098Scraped Successfully!
Page1099Scraped Successfully!
Page1100Scraped Successfully!
Page1101Scraped Successfully!
Page1102Scraped Successfully!
Page1103Scraped Successfully!
Page1104Scraped Successfully!
Page1105Scraped Successfully!
Page1106Scraped Successfully!
Page1107Scraped Successfully!
Page1108Scraped Successfully!
Page1109Scraped Successfully!
Page1110Scraped Successfully!
Page1111Scraped Successfully!
Page1112Scraped Successfully!
Page1113Scraped Successfully!
Page1114Sc

Page1441Scraped Successfully!
Page1442Scraped Successfully!
Page1443Scraped Successfully!
Page1444Scraped Successfully!
Page1445Scraped Successfully!
Page1446Scraped Successfully!
Page1447Scraped Successfully!
Page1448Scraped Successfully!
Page1449Scraped Successfully!
Page1450Scraped Successfully!
Page1451Scraped Successfully!
Page1452Scraped Successfully!
Page1453Scraped Successfully!
Page1454Scraped Successfully!
Page1455Scraped Successfully!
Page1456Scraped Successfully!
Page1457Scraped Successfully!
Page1458Scraped Successfully!
Page1459Scraped Successfully!
Page1460Scraped Successfully!
Page1461Scraped Successfully!
Page1462Scraped Successfully!
Page1463Scraped Successfully!
Page1464Scraped Successfully!
Page1465Scraped Successfully!
Page1466Scraped Successfully!
Page1467Scraped Successfully!
Page1468Scraped Successfully!
Page1469Scraped Successfully!
Page1470Scraped Successfully!
Page1471Scraped Successfully!
Page1472Scraped Successfully!
Page1473Scraped Successfully!
Page1474Sc

Page1801Scraped Successfully!
Page1802Scraped Successfully!
Page1803Scraped Successfully!
Page1804Scraped Successfully!
Page1805Scraped Successfully!
Page1806Scraped Successfully!
Page1807Scraped Successfully!
Page1808Scraped Successfully!
Page1809Scraped Successfully!
Page1810Scraped Successfully!
Page1811Scraped Successfully!
Page1812Scraped Successfully!
Page1813Scraped Successfully!
Page1814Scraped Successfully!
Page1815Scraped Successfully!
Page1816Scraped Successfully!
Page1817Scraped Successfully!
Page1818Scraped Successfully!
Page1819Scraped Successfully!
Page1820Scraped Successfully!
Page1821Scraped Successfully!
Page1822Scraped Successfully!
Page1823Scraped Successfully!
Page1824Scraped Successfully!
Page1825Scraped Successfully!
Page1826Scraped Successfully!
Page1827Scraped Successfully!
Page1828Scraped Successfully!
Page1829Scraped Successfully!
Page1830Scraped Successfully!
Page1831Scraped Successfully!
Page1832Scraped Successfully!
Page1833Scraped Successfully!
Page1834Sc

Page2161Scraped Successfully!
Page2162Scraped Successfully!
Page2163Scraped Successfully!
Page2164Scraped Successfully!
Page2165Scraped Successfully!
Page2166Scraped Successfully!
Page2167Scraped Successfully!
Page2168Scraped Successfully!
Page2169Scraped Successfully!
Page2170Scraped Successfully!
Page2171Scraped Successfully!
Page2172Scraped Successfully!
Page2173Scraped Successfully!
Page2174Scraped Successfully!
Page2175Scraped Successfully!
Page2176Scraped Successfully!
Page2177Scraped Successfully!
Page2178Scraped Successfully!
Page2179Scraped Successfully!
Page2180Scraped Successfully!
Page2181Scraped Successfully!
Page2182Scraped Successfully!
Page2183Scraped Successfully!
Page2184Scraped Successfully!
Page2185Scraped Successfully!
Page2186Scraped Successfully!
Page2187Scraped Successfully!
Page2188Scraped Successfully!
Page2189Scraped Successfully!
Page2190Scraped Successfully!
Page2191Scraped Successfully!
Page2192Scraped Successfully!
Page2193Scraped Successfully!
Page2194Sc

Page2521Scraped Successfully!
Page2522Scraped Successfully!
Page2523Scraped Successfully!
Page2524Scraped Successfully!
Page2525Scraped Successfully!
Page2526Scraped Successfully!
Page2527Scraped Successfully!
Page2528Scraped Successfully!
Page2529Scraped Successfully!
Page2530Scraped Successfully!
Page2531Scraped Successfully!
Page2532Scraped Successfully!
Page2533Scraped Successfully!
Page2534Scraped Successfully!
Page2535Scraped Successfully!
Page2536Scraped Successfully!
Page2537Scraped Successfully!
Page2538Scraped Successfully!
Page2539Scraped Successfully!
Page2540Scraped Successfully!
Page2541Scraped Successfully!
Page2542Scraped Successfully!
Page2543Scraped Successfully!
Page2544Scraped Successfully!
Page2545Scraped Successfully!
Page2546Scraped Successfully!
Page2547Scraped Successfully!
Page2548Scraped Successfully!
Page2549Scraped Successfully!
Page2550Scraped Successfully!
Page2551Scraped Successfully!
Page2552Scraped Successfully!
Page2553Scraped Successfully!
Page2554Sc

Page2881Scraped Successfully!
Page2882Scraped Successfully!
Page2883Scraped Successfully!
Page2884Scraped Successfully!
Page2885Scraped Successfully!
Page2886Scraped Successfully!
Page2887Scraped Successfully!
Page2888Scraped Successfully!
Page2889Scraped Successfully!
Page2890Scraped Successfully!
Page2891Scraped Successfully!
Page2892Scraped Successfully!
Page2893Scraped Successfully!
Page2894Scraped Successfully!
Page2895Scraped Successfully!
Page2896Scraped Successfully!
Page2897Scraped Successfully!
Page2898Scraped Successfully!
Page2899Scraped Successfully!
Page2900Scraped Successfully!
Page2901Scraped Successfully!
Page2902Scraped Successfully!
Page2903Scraped Successfully!
Page2904Scraped Successfully!
Page2905Scraped Successfully!
Page2906Scraped Successfully!
Page2907Scraped Successfully!
Page2908Scraped Successfully!
Page2909Scraped Successfully!
Page2910Scraped Successfully!
Page2911Scraped Successfully!
Page2912Scraped Successfully!
Page2913Scraped Successfully!
Page2914Sc

In [41]:
# create DataFrame
df = pd.DataFrame({'Post Datetime': post_date,
                   'Post Title': post_titles,
                  'Neighborhood': post_cities,
                  'Bedroom': num_bedroom,
                  'SQFT': sqfts,
                  'Price': prices})

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3000 non-null   object 
 1   Post Title     3000 non-null   object 
 2   Neighborhood   3000 non-null   object 
 3   Bedroom        2880 non-null   object 
 4   SQFT           2635 non-null   float64
 5   Price          3000 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 140.8+ KB
None


,Post Datetime,Post Title,Neighborhood,Bedroom,SQFT,Price
0,2022-12-07 11:02,2 Bedroom in Kits,Kits-Vancouver,2,850.0,2700
1,2022-12-07 11:02,Port Moody prime location with three bedroom f...,Center of Port Moody,3,1818.0,138
2,2022-12-07 11:01,FULLY FURNISHED Studio & Den with Private Balc...,Yaletown-Vancouver,NaN,500.0,2600
3,2022-12-07 11:01,One bedroom brandnew basement south slope cl...,Burnaby,1,NaN,1750
4,2022-12-07 11:01,Brand new 3 bedroom apartment available now,6000 McKay ave,3,1200.0,4000


### Process of finding the values:

In [30]:
# Show first post to look into it
post_one = posts[0]
post_one

<li class="result-row" data-pid="7549565748">
<a class="result-image gallery" data-ids="3:00U0U_3oFMKpbvl2Nz_0CI0t2,3:00H0H_9SLg39h8KgXz_0CI0t2,3:00505_1IRbOpXuF4Bz_0CI0t2,3:00i0i_7qt5sbxTrT8z_0CI0t2,3:00u0u_8WyJ2A5GhTjz_0CI0t2,3:00q0q_17ihqtm9v9Nz_0CI0t2" href="https://abbotsford.craigslist.org/apa/d/surrey-northeast-bedroom-detached-suite/7549565748.html">
<span class="result-price">$2,400</span>
</a>
<div class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2022-10-28 14:36" title="Fri 28 Oct 02:36:16 PM">Oct 28</time>
<h3 class="result-heading">
<a class="result-title hdrlnk" data-id="7549565748" href="https://abbotsford.craigslist.org/apa/d/surrey-northeast-bedroom-detached-suite/7549565748.html" id="postid_7549565748">2 Bedroom detached suite</a>
</h3>
<span class="result-meta">
<span class="result-price">$2,400</span>
<span class="housing">
                    2br -

In [37]:
# City posted
post_city = post_one.find('span', class_='result-hood').text.strip('( )')
post_city

'Abbotsford'

In [42]:
# Number of bedroom
num_bed = int(post_one.find('span', class_='housing').text.split()[0].replace('br', ''))
num_bed, type(num_bed)

(5, int)

In [31]:
# SQFT
sqft = int(post_one.find('span', class_='housing').text.split()[2].replace('ft2', ''))
sqft, type(sqft)

(800, int)

In [36]:
# rental price
price = int(post_one.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
price, type(price)

(3695, int)

In [38]:
# Post ad title
post_ad = post_one.find('a', class_='result-title hdrlnk').text
post_ad

'Beautiful East Abbotsford 5 bdrm House for Rent'

In [56]:
# post datetime
datetime = datetime.strptime(post_one.find('time', class_='result-date')['datetime'], '%Y-%m-%d %H:%M')
datetime, type(datetime)

(datetime.datetime(2022, 12, 5, 11, 23), datetime.datetime)

In [23]:
search_legend = soup.find('div', class_='search-legend')
search_legend

<div class="search-legend">
<div class="search-view js-only">
<div aria-expanded="false" aria-label="view options" class="dropdown dropdown-icons dropdown-arrows dropdown-view" role="toolbar">
<ul class="dropdown-list">
<li class="dropdown-item mode sel">
<button data-selection="list" id="listview" title="show results in a list">
<span class="view-list icon"></span>list
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="pic" id="picview" title="show results in a list with thumbnail pictures">
<span class="view-thumb icon"></span>thumb
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="grid" id="gridview" title="show results side-by-side with larger pictures">
<span class="view-gallery icon"></span>gallery
                </button>
<span class="toggle-arrow"></span>
</li>
<li class="dropdown-item mode">
<button data-selection="map" id="mapview" 

In [27]:
# Find out the beggining of the post number 
num_post_start = search_legend.find('span', class_='rangeFrom').text
num_post_start

'1441'

In [59]:
# Find out total number of the search result
results_num = soup.find('div', class_='search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
results_total

1536

## Process

Check the dataset I created from the Craigslist ad and final adjustment of the dataset to analyze the data. 

First to check the dataset:

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  3000 non-null   object 
 1   Post Title     3000 non-null   object 
 2   Neighborhood   3000 non-null   object 
 3   Bedroom        2880 non-null   object 
 4   SQFT           2635 non-null   float64
 5   Price          3000 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 140.8+ KB


I would like to distingish FULLY FURNISHED suite and not furnished suite. I will create another column to show if it isTURE or FALSE. I noticed the title says if it is furnished suite or not (If not on the title, I assume that the suite is not furnished) 

Also, I noticed that the number of bedroom is string, so I will convert to integer. 

## Analyze

## Share

## Act